In [2]:
import numpy as np
import pandas as pd

In [ ]:
from models import mlp, lstm, cnn

X, y = sliding_window_preprocessing(feature_scaling(df, "sum"), 3, 2)
model = lstm(X.shape, horizon=2, return_sequences=True, loss="mse", lstm_layers=[100, 100])
model.fit(X, y, epochs=200)

In [ ]:
inverse_scaling(model.predict(np.array(X_scaler.transform([[70, 75], [80, 85], [90, 95]]).reshape(1, 3, 2))))

In [39]:
def sliding_window_preprocessing(sequences, history_window, horizon=1, target_col=None):
    if target_col:
        target = sequences.pop(target_col)
        sequences[target_col] = target
    sequences = np.array(sequences)
    
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + history_window
        horizon_end_ix = end_ix + horizon - 1
        # check if we are beyond the sequence
        if horizon_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:horizon_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [24]:
lst = [["Store 1", 1, 2, 5],
       ["Store 1", 3, 4, 8],
       ["Store 1", 5, 6, 12],
       ["Store 2", 10, 20, 35],
       ["Store 2", 30, 40, 75],
       ["Store 2", 50, 60, 115]]
df = pd.DataFrame(lst, columns=["Store", "num1", "num2", "res"])

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df["Store"] = le.fit_transform(df["Store"])

In [28]:
from models import mlp, lstm, cnn
X, y = sliding_window_preprocessing(feature_scaling(df, "res"), 2, target_col="res")
model = lstm(X.shape, horizon=1, return_sequences=True, loss="mse", lstm_layers=[100, 100])
model.fit(X, y, epochs=200)

Epoch 1/200
1/1 [==============================] - 0s 997us/step - loss: 0.2940
Epoch 2/200
1/1 [==============================] - 0s 2ms/step - loss: 0.2889
Epoch 3/200
1/1 [==============================] - 0s 3ms/step - loss: 0.2839
Epoch 4/200
1/1 [==============================] - 0s 2ms/step - loss: 0.2789
Epoch 5/200
1/1 [==============================] - 0s 3ms/step - loss: 0.2740
Epoch 6/200
1/1 [==============================] - 0s 2ms/step - loss: 0.2689
Epoch 7/200
1/1 [==============================] - 0s 2ms/step - loss: 0.2636
Epoch 8/200
1/1 [==============================] - 0s 970us/step - loss: 0.2583
Epoch 9/200
1/1 [==============================] - 0s 2ms/step - loss: 0.2527
Epoch 10/200
1/1 [==============================] - 0s 1ms/step - loss: 0.2471
Epoch 11/200
1/1 [==============================] - 0s 2ms/step - loss: 0.2413
Epoch 12/200
1/1 [==============================] - 0s 2ms/step - loss: 0.2353
Epoch 13/200
1/1 [==============================] - 0s 3m

In [34]:
data = [[1, 50, 60],
        [1, 70, 80]]

inverse_scaling(model.predict(np.array(X_scaler.transform(data).reshape(1, 2, 3))))

array([[431.74066]], dtype=float32)

In [60]:
lst = [[1, "Store 1", 1, 2, 5],
       [2, "Store 1", 3, 4, 8],
       [3, "Store 1", 5, 6, 12],
       [1, "Store 2", 10, 20, 35],
       [2, "Store 2", 30, 40, 75],
       [3, "Store 2", 50, 60, 115],
        [1, "Store 1", 100, 2, 5],
       [2, "Store 1", 300, 4, 8],
       [3, "Store 1", 500, 6, 12],
       [1, "Store 2", 1000, 20, 35],
       [2, "Store 2", 3000, 40, 75],
       [3, "Store 2", 5000, 60, 115]]
df = pd.DataFrame(lst, columns=["item", "store", "num1", "num2", "res"])

,store(t-2),num1(t-2),num2(t-2),res(t-2),store(t-1),num1(t-1),num2(t-1),res(t-1),store(t),num1(t),num2(t),res(t),store(t+1),num1(t+1),num2(t+1),res(t+1)
2,Store 1,50.5,2.0,5.0,Store 2,505.0,20.0,35.0,Store 1,151.5,4,8,Store 2,1515.0,40.0,75.0
3,Store 2,505.0,20.0,35.0,Store 1,151.5,4.0,8.0,Store 2,1515.0,40,75,Store 1,252.5,6.0,12.0
4,Store 1,151.5,4.0,8.0,Store 2,1515.0,40.0,75.0,Store 1,252.5,6,12,Store 2,2525.0,60.0,115.0


In [68]:
def sliding_window_with_breakdown(sequences, history_window, horizon=1, breakdown_columns=[], target_col=None):
    if len(breakdown_columns) == 0:
        print("regular sliding window")
        return
    X, y = np.array([]), np.array([])
    cols = sequences[breakdown_columns].drop_duplicates()
    for row in cols.values:
        q = []
        for col_name, val in zip(breakdown_columns, row):
            if type(val) == int or type(val) == float:
                q.append(f"{col_name}=={val}")
            else:
                q.append(f"{col_name}=='{val}'")
             
        q = " and ".join(q)
        temp_df = sequences.query(q).copy()
        X_temp, y_temp = sliding_window_preprocessing(temp_df, history_window, horizon, target_col)
        if X.size == 0:
            X = X_temp.copy()
        else:
            X = np.concatenate((X, X_temp))
        if y.size == 0:
            y = y_temp.copy()
        else:
            y = np.concatenate((y, y_temp))
    
    return X, y
sliding_window_with_breakdown(df, 2, breakdown_columns=['Store', "date"], target_col="Store")

(array([[[1, 1, 2, 5],
         [1, 100, 2, 5]],
 
        [[2, 3, 4, 8],
         [2, 300, 4, 8]],
 
        [[3, 5, 6, 12],
         [3, 500, 6, 12]],
 
        [[1, 10, 20, 35],
         [1, 1000, 20, 35]],
 
        [[2, 30, 40, 75],
         [2, 3000, 40, 75]],
 
        [[3, 50, 60, 115],
         [3, 5000, 60, 115]]], dtype=object),
 array([['Store 1'],
        ['Store 1'],
        ['Store 1'],
        ['Store 2'],
        ['Store 2'],
        ['Store 2']], dtype=object))

In [21]:
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler()
Y_scaler = MinMaxScaler()

def feature_scaling(sequences, target_column):
    feature_columns = df.drop([target_column], axis=1).columns.values
    
    X_data = X_scaler.fit_transform(sequences[feature_columns])
    Y_data = Y_scaler.fit_transform(sequences[target_column].values.reshape(-1, 1))
    
    sequences[feature_columns], sequences[target_column] = X_data, Y_data
    return sequences

def inverse_scaling(y):
    return Y_scaler.inverse_transform(y)

,num1,num2,sum
0,0.000,0.000,0.000
1,0.125,0.125,0.125
2,0.250,0.250,0.250
3,0.375,0.375,0.375
4,0.500,0.500,0.500
5,0.625,0.625,0.625
6,0.750,0.750,0.750
7,0.875,0.875,0.875
8,1.000,1.000,1.000


In [2]:
def train_test_split():
    pass